<h1>This script allows you to read pixel time series of NDVI values from NASA's GIMMs dataset and analyze it using PolyTrend algorithm</h1>
<p>To use this tool, you need to set up an account with Google Earth Engine. Do it <a href'https://earthengine.google.com/signup'> here.</a></p>
<p>For more information about PolyTrend see the original paper <a href='https://www.sciencedirect.com/science/article/pii/S0034425713003878'> here.</a></p>
<p>This script can be adjusted to use any other dataset.</p>

In [4]:
import ipywidgets
#set up authentication credentials. alternatively run the command 'earthengine authenticate' in conda
##run this and wait for browser's response:
# ipywidgets.HTML(
#     '<a href="{url}" target="auth">Open OAuth2 Page</a><br/>'.format(
#         url=ee.oauth.get_authorization_url()
#     )
# )

In [3]:
#copy the athentication code from the browser
# code = '4/UwBKru7-cX3H1pRdAxm_96TZJJtwTSCfWMhMry1GPlmUaqp2E-eGgDk'
# try:
#     token = ee.oauth.request_token(code)
#     ee.oauth.write_token(token)
#     print('Successfully saved authorization token.')
# except Exception as e:
#     print(e)

name 'ee' is not defined


In [1]:
#import libraries for PolyTrend algorithm and downloading the time series data
import pandas as pd
import ee
import numpy as np
import numpy.linalg as lng
import numpy.polynomial.polynomial as poly
import scipy.stats as stats
from ipyleaflet import Map, basemaps, DrawControl, GeoJSON, basemap_to_tiles 
    
ee.Initialize()

In [3]:
#select the image collection and filter dates
name_of_collection = 'NASA/GIMMS/3GV0'
start_date = '2011-12-01'
end_date = '2012-12-02'
#set desired statistical significance for the fit
alpha = 0.05 
#download image collection
collection = ee.ImageCollection(name_of_collection).filterDate(start_date, end_date)


In [4]:
#create a map and create a point to get coordinates. The coordinated of the last marked point will be used for further computation
m = Map(
    center=(52.834502, 13.798697),
    zoom=7
)
#enable drawing tools
draw_control = DrawControl()
draw_control.point = {
    "shapeOptions": {
        "fillColor": "#6be5c3",
        "color": "#6be5c3",
        "fillOpacity": 1.0
    }
}

#mark a point to set area of interest
m.add_control(draw_control)
m


Map(basemap={'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors',…

In [9]:
coords = draw_control.last_draw['geometry']['coordinates']
print(coords)

def GetDataFrame(coords):
    
    pnt = ee.Geometry.Point(coords)
    # Sample for a time series of values at the point. After 'select' enter which bands are needed
    #scale is meters for each pixel. if the number is too low, there will be too many records
    geom_values = collection.filterBounds(pnt).getRegion(geometry=pnt, scale=700)
    geom_values_list = ee.List(geom_values).getInfo()
    # Convert to a Pandas DataFrame.
    header = geom_values_list[0]
    data = pd.DataFrame(geom_values_list[1:], columns=header)
    data['datetime'] = pd.to_datetime(data['time'], unit='ms', utc=True)
    data.set_index('time')
    data = data.sort_values('time')
    data = data['ndvi']
    #change pandas data frame into numpy array with.values
    data = data.values
    return data;

#Get time series for the point of interest

Y = GetDataFrame(coords)


[13.44956, 53.164558]
[0.43700001 0.38100001 0.463      0.58499998 0.60000002 0.64099997
 0.66399997 0.72899997 0.76200002 0.84600002 0.76700002 0.773
 0.85699999 0.90899998 0.80599999 0.72000003 0.736      0.78299999
 0.72799999 0.764      0.86699998 0.83099997 0.63999999 0.49599999
 0.477      0.47       0.51999998 0.58200002 0.66799998 0.67500001
 0.685      0.64300001 0.69       0.73400003 0.829      0.764
 0.72799999 0.634      0.713      0.76999998 0.73400003 0.61199999
 0.58200002 0.722      0.81099999 0.79400003 0.58600003 0.50099999
 0.546      0.55000001 0.41499999 0.368      0.454      0.59500003
 0.57300001 0.50599998 0.57300001 0.74299997 0.80699998 0.83399999
 0.84500003 0.83200002 0.82999998 0.78500003 0.77700001 0.75599998
 0.73900002 0.755      0.81099999 0.75300002 0.68599999 0.63700002
 0.61000001]


In [10]:
#PolyTrend algorithm that returns the type of trend, statistical significance of it and polynomial degree 

def PolyTrend(Y, alpha):
    X = range(1, len(Y)+1)
    
    #define function to find p value:
    def Pvalue(coef, df, A, Aprim, pn):
        #generate square residual
        part_res = np.dot(A, pn)-Y
        residual = np.dot(part_res.transpose(), part_res)
        #generate variance-covariance matrix
        VC = lng.inv(np.dot(Aprim, A))*residual/df
        #compute variance of the first coefficient
        VC1 = np.sqrt(VC[0,0])
        #compute t-statistic
        statistic = coef/VC1
        #compute p value
        p = stats.t.sf(np.abs(statistic), df)*2
        return p;

    def Plinear(X, Y):
        degree = 1
        df1 = len(X)-degree-1
        #generate Vandermonde matrix
        A1 = np.vander(X, 2)
        #generate transpose of the Vandermonde matrix
        Aprim1 = A1.transpose()
        p1 = np.dot(np.dot((lng.inv(np.dot(Aprim1, A1))), Aprim1), Y)
        coef1 = p1[0]
        Plin = Pvalue(coef1, df1, A1, Aprim1, p1)
        Slope = p1[0]
        Direction = np.sign(Slope)
        return Plin;
    
    degree = 3
    #degrees of freedom
    df3 = len(X)-degree-1
    #generate Vandermonde matrix
    A3 = np.vander(X, 4)
    #generate transpose of the Vandermonde matrix
    Aprim3 = A3.transpose()
    #X=inv(A'*A)*A'*L - creating coefficients matrix:
    p3 = np.dot(np.dot((lng.inv(np.dot(Aprim3, A3))), Aprim3), Y)
    coef3 = p3[0]
    #compute p-value for cubic fit
    Pcubic = Pvalue(coef3, df3, A3, Aprim3, p3)
    #get roots of cubic polynomial
    coefs3 = ([p3[2], 2*p3[1], 3*p3[0]])
    roots3 = np.sort(poly.polyroots(coefs3))

    if (np.imag(roots3[0]) == 0 and np.imag(roots3[1])==0 and roots3[0] != roots3[1] and X[0] <= roots3[0] and roots3[0] <= len(X) and X[0] <= roots3[1] and roots3[1] <= len(X) and Pcubic < alpha):
        Plin = Plinear(X, Y)
        if (Plin < alpha):
            Trend_type = 3
            Significance = 1
            Poly_degree = 3
        else:
            Trend_type = -1
            Significance = -1
            Poly_degree = 3
            return 'trend type:', trend_name, 'significance:', Significance, 'polynomial degree:', Poly_degree;
    else:
        degree = 2
        df2 = len(X)-degree-1
        A2 = np.vander(X, 3)
        Aprim2 = A2.transpose()
        p2 = np.dot(np.dot((lng.inv(np.dot(Aprim2, A2))), Aprim2), Y)
        coef2 = p2[0]
        Pquadratic = Pvalue(coef2, df2, A2, Aprim2, p2)
        coefs2 = ([p2[1], 2*p2[0]])
        roots2 = np.sort(poly.polyroots(coefs2))
        
        if (1<=roots2 and roots2 <= len(X) and Pquadratic < alpha):
            Plin = Plinear(X, Y)
            if Plin < alpha:
                Trend_type = 2
                Significance = 1
                Poly_degree = 2
            else:
                Trend_type = -1
                Significance = -1
                Poly_degree = 2
                return 'trend type:', Trend_type, 'significance:', Significance, 'polynomial degree:', Poly_degree;
                
        else:
            Plin = Plinear(X, Y)
            if Plin < alpha:
                Trend_type = 1
                Significance = 1
                Poly_degree = 1
            else:
                Trend_type = 0
                Significance = -1
                Poly_degree = 0
            return 'trend type:', Trend_type, 'significance:', Significance, 'polynomial degree:', Poly_degree;
        return 'trend type:', Trend_type, 'significance:', Significance, 'polynomial degree:', Poly_degree;
    return;

PolyTrend(Y, alpha)     


C:\Users\Lenovo\Anaconda2\envs\polytrend\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in sqrt
C:\Users\Lenovo\Anaconda2\envs\polytrend\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\Lenovo\Anaconda2\envs\polytrend\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\Lenovo\Anaconda2\envs\polytrend\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


('trend type:', 0, 'significance:', -1, 'polynomial degree:', 0)